In [5]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

pd.set_option('display.max_columns', None)

#import geopandas as gpd
#from geopy.exc import GeocoderTimedOut 
#from geopy.geocoders import Nominatim 
#from opencage.geocoder import OpenCageGeocode

#key = 'e7fc4f62e32840878a466fb71e2ebda1'
#geocoder = OpenCageGeocode(key)

import re
import sklearn

In [6]:
df = pd.read_csv('alldata.csv')

In [7]:
df.shape

(6964, 5)

In [8]:
# check if dataframe has NaN
df.isna().sum()

position         11
company          11
description      11
reviews        1638
location         11
dtype: int64

In [9]:
# remove NaN rows
df = df[df['position'].notna()]

In [10]:
# check that all NaN are gone
df.isna().sum()

position          0
company           0
description       0
reviews        1627
location          0
dtype: int64

In [12]:
#split location into state and city
df[['City', 'State']] = df['location'].str.split(',', 1, expand=True)

In [13]:
#make new column for zip code
df['Zip-code']=df.location.str.extract('(\d+)')

In [14]:
# delete numbers in State
df['State'] = df['State'].str.replace('\d+', '')

In [15]:
#delete original location column
df.drop(['location'], axis=1)

,position,company,description,reviews,City,State,Zip-code
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,NaN,Atlanta,GA,30301
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",NaN,Atlanta,GA,NaN
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",NaN,Atlanta,GA,NaN
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,44.0,Atlanta,GA,30303
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,550.0,Atlanta,GA,NaN
...,...,...,...,...,...,...,...
6959,Data Developer / Machine Learning Analyst,NetApp,Are you data-driven? We at NetApp believe in t...,574.0,Sunnyvale,CA,NaN
6960,Scientist I,"Pharmacyclics, an Abbvie Company",Pharmacyclics is committed to the development ...,26.0,Sunnyvale,CA,NaN
6961,Intern Scientist,Oath Inc,"Oath, a subsidiary of Verizon, is a values-led...",5.0,Sunnyvale,CA,NaN
6962,Senior Data & Applied Scientist,Microsoft,We are the Bing Core Relevance team responsibl...,4618.0,Sunnyvale,CA,NaN


In [17]:
#deleting the old 'location' column
df=df.drop(columns=['location'])

In [18]:
# rename state with initials
df['State'] = df['State'].str.replace('^ +| +$', '')

In [19]:
#replace NaN with 0 in reviews
df['reviews']=df['reviews'].fillna(0)

In [20]:
# checking what kind of words to look for to extract seniority
df[df['position'].str.contains("Senior").fillna(False)]

,position,company,description,reviews,City,State,Zip-code
9,Senior Associate - Cognitive Data Scientist Na...,KPMG,Known for being a great place to work and buil...,4494.0,Atlanta,GA,30338
12,"Senior Associate, Data Scientist",KPMG,Innovate. Collaborate. Shine. Lighthouse — KPM...,4494.0,Atlanta,GA,30338
16,Senior Structural Engineer-Transmission & Dist...,Burns & McDonnell,Description\nExecutes complex assignments requ...,152.0,Atlanta,GA,30309
17,Senior Mobile Developer (iOS/Android),Perficient,Overview\n\n\n\nAt Perficient you’ll deliver m...,44.0,Atlanta,GA,30338
20,Senior Electrical Engineer,Burns & McDonnell,Description\nBurns &amp; McDonnell is consider...,152.0,Atlanta,GA,30309
...,...,...,...,...,...,...,...
6915,Senior/Staff Full Stack Engineer - Advertising...,Walmart eCommerce,Position Description\nWe are seeking talented ...,141.0,Sunnyvale,CA,NaN
6916,"Senior Manager, Cost Accounting","Pharmacyclics, an Abbvie Company",Pharmacyclics is committed to the development ...,26.0,Sunnyvale,CA,NaN
6923,Senior Data Scientist,Castlight Health,We’re hiring data scientists for a team of dat...,7.0,Sunnyvale,CA,94086
6958,Senior Data Scientist (GEC11902),Walmart,Position Description\nProficient in Java or Py...,148114.0,Sunnyvale,CA,94086


In [21]:
#isolating the different levels of seniority
df.loc[df['position'].str.contains('Junior|Jr|junior|jr', case=False), 'seniority'] = 'junior'
df.loc[df['position'].str.contains('Senior|Sr|senior|sr', case=False), 'seniority'] = 'senior'
df.loc[df['position'].str.contains('Entry|entry', case=False), 'seniority'] = 'entry level'

df.seniority.value_counts()

senior         1486
junior           49
entry level      17
Name: seniority, dtype: int64

In [38]:
#isolating the different role types
df.loc[df['position'].str.contains('Scientist|scientist|science|Science', case=False), 'role'] = 'data scientist'
df.loc[df['position'].str.contains('Analyst|analyst', case=False), 'role'] = 'data analyst'
df.loc[df['position'].str.contains('Engineer|engineer|Engineering|engineering', case=False), 'role'] = 'data engineer'
df.loc[df['position'].str.contains('Developer|developer', case=False), 'role'] = 'developer'
df.loc[df['position'].str.contains('Manager|manager', case=False), 'role'] = 'manager'
df.loc[df['position'].str.contains('Director|director', case=False), 'role'] = 'director'


df.role.value_counts()

data scientist    2499
data engineer     1294
data analyst       852
manager            549
director           261
developer          128
Name: role, dtype: int64

In [39]:
# checking how many NaN these columns have (too many)
df.isna().sum()

position          0
company           0
description       0
reviews        1627
location          0
seniority      5401
role           1370
dtype: int64

### What should we do with fields that have more than one of these terms? 
#### For example 'Data science and analytics'

In [78]:
# create columns for each skills that could be found in the description
df['sql'] = np.where(df['description'].str.contains('SQL'or'sql'),'sql','')
df['python'] = np.where(df['description'].str.contains('python'or'Python'or'PYTHON'),'python','')
df['R'] = np.where(df['description'].str.contains('R'),'R','')
#df['C++'] = np.where(df['description'].str.contains('C++'or'c++''),"c++",'')
df['ML'] = np.where(df['description'].str.contains('machine learning'or'Machine learning'or'Machine Learning'or'ML'),'ML','')
df.python.value_counts()

In [74]:
# concatenate all these columns into one names skills
df['skills']=df['sql']+' ,'+df['python']+' ,'+df['R']+' ,'+df['ML']

In [79]:
df['skills'] = df[['sql', 'python','R','ML']].agg(','.join, axis=1)
df


,position,company,description,reviews,location,seniority,role,sql,python,R,ML,skills
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,NaN,"Atlanta, GA 30301",NaN,director,,,R,,",,R,"
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",NaN,"Atlanta, GA",NaN,data scientist,,,R,,",,R,"
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",NaN,"Atlanta, GA",NaN,data scientist,sql,,R,,"sql,,R,"
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,44.0,"Atlanta, GA 30303",NaN,data analyst,sql,,R,,"sql,,R,"
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,550.0,"Atlanta, GA",NaN,NaN,,,R,ML,",,R,ML"
...,...,...,...,...,...,...,...,...,...,...,...,...
6959,Data Developer / Machine Learning Analyst,NetApp,Are you data-driven? We at NetApp believe in t...,574.0,"Sunnyvale, CA",NaN,developer,sql,,R,ML,"sql,,R,ML"
6960,Scientist I,"Pharmacyclics, an Abbvie Company",Pharmacyclics is committed to the development ...,26.0,"Sunnyvale, CA",NaN,data scientist,,,R,,",,R,"
6961,Intern Scientist,Oath Inc,"Oath, a subsidiary of Verizon, is a values-led...",5.0,"Sunnyvale, CA",NaN,data scientist,sql,,R,ML,"sql,,R,ML"
6962,Senior Data & Applied Scientist,Microsoft,We are the Bing Core Relevance team responsibl...,4618.0,"Sunnyvale, CA",senior,data scientist,,,R,ML,",,R,ML"


In [23]:
# extract the education level from the description field
def CleanDescriptionQual (x):
    if 'bsc' in x.lower():
        return 'Bachelor'
    elif 'bachelor' in x.lower():
        return 'Bachelor'
    elif 'msc' in x.lower():
        return 'Masters'
    elif 'masters' in x.lower():
        return 'Masters'
    elif 'master' in x.lower():
        return 'Masters'
    elif 'phd' in x.lower():
        return 'PhD'
    else:
        return 'Not specified'
    
df['education'] = df['description'].apply(CleanDescriptionQual)

In [24]:
df

,position,company,description,reviews,City,State,Zip-code,seniority,education
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,0.0,Atlanta,GA,30301,NaN,Bachelor
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",0.0,Atlanta,GA,NaN,NaN,Masters
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",0.0,Atlanta,GA,NaN,NaN,Masters
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,44.0,Atlanta,GA,30303,NaN,Bachelor
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,550.0,Atlanta,GA,NaN,NaN,PhD
...,...,...,...,...,...,...,...,...,...
6959,Data Developer / Machine Learning Analyst,NetApp,Are you data-driven? We at NetApp believe in t...,574.0,Sunnyvale,CA,NaN,NaN,Bachelor
6960,Scientist I,"Pharmacyclics, an Abbvie Company",Pharmacyclics is committed to the development ...,26.0,Sunnyvale,CA,NaN,NaN,Bachelor
6961,Intern Scientist,Oath Inc,"Oath, a subsidiary of Verizon, is a values-led...",5.0,Sunnyvale,CA,NaN,NaN,PhD
6962,Senior Data & Applied Scientist,Microsoft,We are the Bing Core Relevance team responsibl...,4618.0,Sunnyvale,CA,NaN,senior,Not specified


In [25]:
df.to_csv(r'cleandata.csv', index = False)